In [2]:
import os
from distutils.dir_util import copy_tree

import numpy as np
import cv2

import tensorflow as tf
from tensorflow.keras.layers import GlobalMaxPooling2D, Input

from tqdm import tqdm
import pickle
# os.mkdir('Dataset') 

In [3]:
#Dataset from kaggle https://www.kaggle.com/datasets/vikashrajluhaniwal/fashion-images

data_dir = 'Dataset'
boy_app = 'data/Apparel/Boys/Images/images_with_product_ids'
gir_app = 'data/Apparel/Girls/Images/images_with_product_ids'
men_foot = 'data/Footwear/Men/Images/images_with_product_ids'
wom_foot = 'data/Footwear/Women/Images/images_with_product_ids'

In [4]:
data_list = [boy_app, gir_app, men_foot, wom_foot]

for i in data_list:
    copy_tree(i, data_dir)

In [5]:
feature_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False,
                                                       input_shape=(224,224,3))
feature_model.trainable = False

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

94765736/94765736 [==============================] - 24s 0us/step


In [6]:
preprocess_layer = tf.keras.applications.resnet50.preprocess_input

inputs = Input(shape=(224,224,3))
x = preprocess_layer(inputs)
x = feature_model(x)
output = GlobalMaxPooling2D()(x)

model = tf.keras.Model(inputs, output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable pa

In [7]:
def extract(img_path, model):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224,224))
    img = np.array(img)
    img = np.expand_dims(img, 0)
    pred = model.predict(img, verbose=0).flatten()
    return pred/np.linalg.norm(pred)

In [1]:
data_array = []
for i in tqdm(os.listdir(data_dir)):
    vector = extract(os.path.join(data_dir, i), model)
    data_array.append(vector)
    
data_array = np.array(data_array)

In [ ]:
pickle.dump(data_array, open('f_vectors.pkl', 'wb'))
pickle.dump(os.listdir(data_dir), open('filenames.pkl', 'wb'))